<a href="https://colab.research.google.com/github/shahzad-r1zv1/LocalLLM_experiments/blob/main/7S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install -q pandas tqdm matplotlib reportlab python-pptx requests


In [ ]:
# ============================== ONE-CELL 7S + OLLAMA (COLAB) ==============================
# Installs Ollama, pulls up to 3 models, runs selected 7S prompts across models,
# consolidates results, and saves JSON/Markdown outputs in /content.

import os, subprocess, shlex, time, requests, json, re, textwrap, datetime
from typing import Dict, List

# ---------------------------
# 0) BASIC CONFIG (EDIT THESE)
# ---------------------------
# Organization inputs used to fill prompt placeholders
ORG = {
    "name": "QA Automation Org",
    "industry": "Software / DevTools",
    "size_employees": 180,
    "annual_revenue": "N/A (cost center)",
    "locations": "Toronto; Remote",
    "current_context": "Transitioning from phased QA to DevSecOps; aiming for predictive quality and CI/CD-first culture.",
    "key_challenges": "Inconsistent test coverage; siloed defect data; unclear decision rights; skills gap in pipeline engineering.",
    "recent_changes": "Trunk-based dev in 2 squads; feature flags/canary pilots; Playwright + contract tests on 2 critical APIs.",
    # Optional fields for specific prompts (leave as "" if unknown)
    "strategy_statement": "Ship faster with confidence via automated quality gates and proactive risk analytics.",
    "strategic_goals": "12m: 70% automated coverage; 24m: 95% critical path; MTTR<1hr; DORA elite.",
    "kpis": "DORA metrics, escaped defect rate, mean time to detect, e2e reliability SLOs.",
    "market_position": "Internal enablement platform within a large enterprise.",
    "competitive_advantages": "Domain expertise; test data virtualization; unified telemetry.",
    "target_segments": "Product teams building microservices/APIs.",
    "value_proposition": "Fewer incidents, faster releases, higher trust in automation.",
    "org_chart": "VP Eng → Dir QA/Platform → QA Enablement, SDET, Observability pods; squads map to product lines.",
    "layers": "VP → Director → Managers → ICs (4 layers)",
    "span_of_control": "Managers: 6-8 ICs avg",
    "decision_rights": "Product owns scope; QA owns quality standards; Dev owns implementation; shared release gates.",
    "xfunc": "Release Council; Reliability Guild; Architecture Forum.",
    "geo": "Toronto hub + distributed remote",
    "restructuring": "Evolving towards platform team model; decoupling test infra from app squads."
}

# Which prompts to run (choose from keys below)
RUN_PROMPTS = [
    "PROMPT_1_FULL_7S",
    "PROMPT_2_STRATEGY",
    "PROMPT_3_STRUCTURE",
    "PROMPT_4_SYSTEMS",
    "PROMPT_5_SHARED_VALUES",
    "PROMPT_6_SKILLS",
    "PROMPT_7_STYLE",
    "PROMPT_8_STAFF",
    "PROMPT_9_ALIGNMENT",
    "PROMPT_10_CHANGE",
    "PROMPT_11_DIGITAL",
    "PROMPT_12_BENCH",
    "PROMPT_13_GAPS",
    "PROMPT_14_INTEGRATION",
    "PROMPT_15_EXEC_SUMMARY",
    "MEGA_PROMPT"
]

# Models to try pulling (we’ll use what actually succeeds)
REQUESTED_MODELS = ["mistral", "llama3", "phi3"]

# Output paths
OUT_DIR = "/content"
BUNDLE_JSON = os.path.join(OUT_DIR, "7S_bundle.json")
REPORT_MD   = os.path.join(OUT_DIR, "7S_report.md")

# ---------------------------
# 1) INSTALL & START OLLAMA
# ---------------------------
def run(cmd, check=True):
    if isinstance(cmd, str):
        cmd = shlex.split(cmd)
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    if check and p.returncode != 0:
        raise RuntimeError(f"CMD FAIL: {' '.join(cmd)}\n----\n{p.stdout}")
    return p.stdout

print("🔧 Installing Ollama…")
run("bash -lc 'curl -fsSL https://ollama.com/install.sh | sh'", check=False)

print("🚀 Starting ollama serve…")
# kill any prior
run("bash -lc 'pkill -f \"ollama serve\" || true'", check=False)
logf = open("/tmp/ollama.log","w")
proc = subprocess.Popen(["ollama","serve"], stdout=logf, stderr=subprocess.STDOUT, text=True)

BASE = "http://127.0.0.1:11434"
TAGS = f"{BASE}/api/tags"
CHAT = f"{BASE}/api/chat"
GEN  = f"{BASE}/api/generate"

print("⏳ Waiting for Ollama to become ready", end="")
ready=False
for _ in range(60):
    try:
        r = requests.get(TAGS, timeout=3)
        if r.status_code == 200:
            ready=True; break
    except Exception: pass
    print(".", end="")
    time.sleep(2)
print()
if not ready:
    print("⚠️ Ollama did not start. Last log lines:")
    try:
        print("".join(open("/tmp/ollama.log","r",errors="ignore").readlines()[-60:]))
    except: pass
    raise SystemExit("Cannot continue without Ollama.")

# ---------------------------
# 2) PULL MODELS (best-effort)
# ---------------------------
print("📥 Pulling models (best-effort):", ", ".join(REQUESTED_MODELS))
available = []
for m in REQUESTED_MODELS:
    try:
        out = run(["ollama","pull",m], check=False)
        # verify presence
        models = requests.get(TAGS, timeout=10).json().get("models", [])
        roots  = {k.get("name","").split(":")[0] for k in models}
        if m in roots:
            available.append(m)
            print(f"  ✅ {m} ready")
        else:
            print(f"  ⚠️ {m} not listed after pull, skipping")
    except Exception as e:
        print(f"  ⚠️ pull {m} failed: {e}")

if not available:
    raise SystemExit("❌ No models available. Try a smaller one like 'mistral'.")

PRIMARY_MODEL = available[0]
print("✅ Using models:", ", ".join(available), f"(primary: {PRIMARY_MODEL})")

# Probe endpoints: prefer /api/chat, fallback to /api/generate
USE_GENERATE = False
try:
    pr = requests.post(CHAT, json={"model":PRIMARY_MODEL,"messages":[{"role":"user","content":"ping"}],"stream":False}, timeout=10)
    if pr.status_code == 404:
        gr = requests.post(GEN, json={"model":PRIMARY_MODEL,"prompt":"ping","stream":False}, timeout=10)
        if gr.status_code == 200:
            USE_GENERATE=True
            print("ℹ️ Using /api/generate (no /api/chat).")
        else:
            raise RuntimeError(f"generate status {gr.status_code}")
    elif pr.status_code != 200:
        raise RuntimeError(f"chat status {pr.status_code}")
except Exception:
    gr = requests.post(GEN, json={"model":PRIMARY_MODEL,"prompt":"ping","stream":False}, timeout=10)
    if gr.status_code == 200:
        USE_GENERATE=True
        print("ℹ️ Using /api/generate (chat failed).")
    else:
        raise SystemExit("❌ Neither /api/chat nor /api/generate responded OK.")

# ---------------------------
# 3) PROMPT TEMPLATES (from doc)
# ---------------------------
def fill(s: str, kv: Dict[str,str]) -> str:
    # simple replacement of [FIELD] with kv.get(field_lower)
    def rep(m):
        key = m.group(1).strip().lower().replace(" ", "_")
        return str(kv.get(key, f"N/A: {m.group(0)}"))
    return re.sub(r"\[([^\]]+)\]", rep, s)

PROMPTS = {
"PROMPT_1_FULL_7S": """Act as a McKinsey consultant conducting a comprehensive 7S analysis.
INPUTS NEEDED:
- Company/Organization name: [NAME]
- Industry: [INDUSTRY]
- Company size (employees): [NUMBER]
- Annual revenue: [REVENUE]
- Geographic presence: [LOCATIONS]
- Current business context: [BRIEF DESCRIPTION]
- Key challenges facing the organization: [LIST 3-5]
- Recent major changes or initiatives: [DESCRIBE]
Analyze all seven elements:
1. Strategy (competitive positioning, strategic priorities, value proposition)
2. Structure (org design, reporting lines, decision rights)
3. Systems (processes, IT systems, workflows)
4. Shared Values (culture, core beliefs, mission/vision)
5. Style (leadership approach, management practices)
6. Staff (workforce composition, capabilities, engagement)
7. Skills (organizational competencies, competitive advantages)
OUTPUT FORMAT:
- Executive summary with key findings
- Detailed analysis of each S (current state assessment)
- Interconnections and dependencies between elements
- Alignment score (1-10) for each element pair
- Top 5 misalignments requiring attention
- Recommendations prioritized by impact and feasibility
""",
"PROMPT_2_STRATEGY": """Evaluate the clarity and effectiveness of our organizational strategy using the McKinsey 7S model.
INPUTS NEEDED:
- Current strategy statement: [PROVIDE]
- Strategic goals (3-5 year): [LIST]
- Key performance indicators: [LIST]
- Market position: [DESCRIBE]
- Competitive advantages: [LIST]
- Target customer segments: [DESCRIBE]
- Value proposition: [STATEMENT]
Assess our strategy by examining:
1. Clarity and communication throughout organization
2. Alignment with market opportunities
3. Differentiation from competitors
4. Resource allocation alignment
5. Measurability and tracking mechanisms
6. Connection to other 6 S elements
OUTPUT:
- Strategy effectiveness score (1-10) with justification
- Strengths and weaknesses analysis
- Gaps between stated and actual strategy
- Impact assessment on other 6 S elements
- 5 specific recommendations to strengthen strategy
- Communication plan to improve strategy understanding
""",
"PROMPT_3_STRUCTURE": """Analyze our organizational structure's alignment with strategic goals.
INPUTS NEEDED:
- Current org chart: [DESCRIBE HIERARCHY]
- Number of management layers: [NUMBER]
- Span of control averages: [NUMBERS]
- Decision-making authority levels: [DESCRIBE]
- Cross-functional teams/committees: [LIST]
- Geographic/divisional structure: [DESCRIBE]
- Recent restructuring efforts: [DESCRIBE IF ANY]
Evaluate:
1. Structure-strategy fit
2. Decision-making speed and effectiveness
3. Communication flow efficiency
4. Collaboration barriers
5. Duplication or gaps in responsibilities
6. Flexibility for future growth
OUTPUT:
- Structure effectiveness rating with evidence
- Organizational design recommendations
- Proposed org chart modifications
- Impact analysis on systems and staff
- Implementation roadmap for structural changes
- Risk assessment of proposed changes
""",
"PROMPT_4_SYSTEMS": """Assess the effectiveness of operational and management systems.
INPUTS NEEDED:
- Core business processes: [LIST]
- IT systems and platforms: [LIST]
- Performance management systems: [DESCRIBE]
- Financial/budgeting processes: [DESCRIBE]
- Quality control systems: [DESCRIBE]
- Communication systems: [DESCRIBE]
- Decision-making processes: [DESCRIBE]
- Knowledge management systems: [DESCRIBE]
Analyze:
1. Process efficiency and effectiveness
2. System integration and data flow
3. Automation opportunities
4. Performance tracking capabilities
5. User satisfaction and adoption
6. Alignment with strategic objectives
OUTPUT:
- Systems maturity assessment (1-5 scale per system)
- Critical system gaps and redundancies
- Process optimization opportunities
- Technology upgrade recommendations
- Implementation priority matrix
- ROI estimates for system improvements
""",
"PROMPT_5_SHARED_VALUES": """Identify and evaluate core shared values driving culture and decision-making.
INPUTS NEEDED:
- Stated mission/vision/values: [PROVIDE]
- Employee survey results: [KEY FINDINGS]
- Leadership behaviors observed: [EXAMPLES]
- Decision-making patterns: [DESCRIBE]
- Customer feedback themes: [SUMMARIZE]
- Internal communication samples: [PROVIDE EXAMPLES]
- Recognition and reward criteria: [LIST]
Examine:
1. Stated vs. lived values gap analysis
2. Values alignment across hierarchy
3. Values impact on behaviors
4. Cultural strengths and toxicities
5. Values-strategy alignment
6. Employee value perception
OUTPUT:
- Core values identification (top 5 actual vs. stated)
- Cultural health score with supporting evidence
- Values-behavior alignment matrix
- Cultural transformation requirements
- Values reinforcement action plan
- Measurement framework for cultural change
""",
"PROMPT_6_SKILLS": """Evaluate critical skills and competencies across the organization.
INPUTS NEEDED:
- Current workforce skills inventory: [CATEGORIES]
- Strategic capability requirements: [LIST]
- Competitor capabilities: [BENCHMARK DATA]
- Training and development programs: [DESCRIBE]
- Performance review data: [KEY METRICS]
- Skills gaps identified by managers: [LIST]
- Future skill requirements (3-5 years): [ANTICIPATE]
Assess:
1. Current vs. required skills gaps
2. Core competency strengths
3. Competitive skill advantages/disadvantages
4. Skills development effectiveness
5. Knowledge transfer mechanisms
6. Innovation and adaptation capabilities
OUTPUT:
- Skills heat map (current vs. required)
- Critical skills gap analysis with risk levels
- Competency development roadmap
- Make/buy/partner talent decisions
- L&D investment recommendations
- Skills KPIs and tracking mechanisms
""",
"PROMPT_7_STYLE": """Analyze the dominant leadership style and its impact on performance.
INPUTS NEEDED:
- Leadership team composition: [DESCRIBE]
- Leadership assessment data: [IF AVAILABLE]
- Employee engagement scores: [PROVIDE]
- Decision-making examples: [3-5 CASES]
- Communication patterns: [DESCRIBE]
- Change management approaches: [EXAMPLES]
- Succession planning status: [DESCRIBE]
Evaluate:
1. Predominant leadership styles
2. Leadership effectiveness metrics
3. Style-strategy alignment
4. Leadership impact on culture
5. Decision-making patterns
6. Leadership development needs
OUTPUT:
- Leadership style profile with strengths/weaknesses
- Leadership effectiveness score (1-10)
- Style-situation fit analysis
- Leadership development priorities
- Succession planning recommendations
- Leadership behavior change roadmap
""",
"PROMPT_8_STAFF": """Review workforce composition, recruitment, and retention strategies.
INPUTS NEEDED:
- Total headcount and demographics: [PROVIDE]
- Organizational structure by function: [BREAKDOWN]
- Turnover rates by level/function: [DATA]
- Time-to-fill metrics: [AVERAGES]
- Employee engagement scores: [PROVIDE]
- Compensation benchmarking: [POSITION VS MARKET]
- Talent pipeline status: [DESCRIBE]
- Diversity metrics: [PROVIDE]
Analyze:
1. Workforce composition vs. strategic needs
2. Talent acquisition effectiveness
3. Retention risks and drivers
4. Engagement and productivity levels
5. Diversity, equity, and inclusion status
6. Workforce planning adequacy
OUTPUT:
- Workforce health scorecard
- Critical talent risks and mitigation plans
- Recruitment strategy optimization
- Retention program enhancements
- Workforce planning recommendations
- HR metrics dashboard design
""",
"PROMPT_9_ALIGNMENT": """Evaluate how well all seven elements of the McKinsey 7S Framework align.
INPUTS NEEDED:
- Brief assessment of each S element: [PROVIDE STATUS]
- Recent organizational changes: [LIST]
- Performance metrics trends: [LAST 2 YEARS]
- Strategic priorities: [TOP 5]
- Known pain points: [DESCRIBE]
- Success stories: [EXAMPLES]
Assess:
1. Element interdependencies and conflicts
2. Alignment scoring for each element pair (21 combinations)
3. Reinforcing vs. conflicting relationships
4. Impact of misalignments on performance
5. Root cause analysis of gaps
6. Synergy opportunities
OUTPUT:
- 7S alignment matrix with scores
- Critical misalignment identification
- Dependency map visualization
- Prioritized realignment initiatives
- Change sequencing recommendations
- Alignment monitoring framework
""",
"PROMPT_10_CHANGE": """Use the 7S Framework to analyze organizational readiness for change.
INPUTS NEEDED:
- Planned change initiative: [DESCRIBE]
- Change timeline and scope: [PROVIDE]
- Previous change efforts: [SUCCESS/FAILURE EXAMPLES]
- Stakeholder groups affected: [LIST]
- Current change capability maturity: [ASSESS 1-5]
- Resource availability: [BUDGET/PEOPLE]
- Risk tolerance: [LOW/MEDIUM/HIGH]
Analyze each S element for:
1. Current state change readiness
2. Required changes per element
3. Resistance points and drivers
4. Change capability gaps
5. Success enablers and barriers
6. Change impact assessment
OUTPUT:
- Change readiness score by S element
- Resistance heat map
- Change impact assessment matrix
- Stakeholder engagement strategy
- Change roadmap with quick wins
- Risk mitigation plan
- Success metrics framework
""",
"PROMPT_11_DIGITAL": """Apply the 7S Framework to assess digital transformation readiness and impact.
INPUTS NEEDED:
- Current digital maturity: [ASSESS 1-5]
- Digital strategy/initiatives: [DESCRIBE]
- Technology infrastructure: [CURRENT STATE]
- Digital skills inventory: [ASSESS]
- Data and analytics capabilities: [DESCRIBE]
- Customer digital expectations: [SUMMARIZE]
- Competitor digital positioning: [BENCHMARK]
Evaluate digital impact on:
1. Strategy (digital business models)
2. Structure (agile organization needs)
3. Systems (technology architecture)
4. Shared Values (digital culture)
5. Style (digital leadership)
6. Staff (digital talent)
7. Skills (digital capabilities)
OUTPUT:
- Digital maturity assessment by S element
- Digital transformation gaps and priorities
- Technology investment recommendations
- Digital culture transformation plan
- Reskilling/upskilling requirements
- Digital KPIs and governance model
- Transformation roadmap with milestones
""",
"PROMPT_12_BENCH": """Compare our 7S profile against key competitors.
INPUTS NEEDED:
- Top 3-5 competitors: [LIST]
- Competitive intelligence available: [SUMMARIZE]
- Industry best practices: [DESCRIBE]
- Our performance vs. competitors: [METRICS]
- Competitive advantages/disadvantages: [LIST]
- Market position: [DESCRIBE]
Benchmark:
1. Strategy effectiveness comparison
2. Organizational agility assessment
3. Operational excellence comparison
4. Cultural differentiation analysis
5. Leadership capability comparison
6. Talent competitiveness evaluation
7. Innovation capability assessment
OUTPUT:
- Competitive 7S comparison matrix
- Competitive advantage/disadvantage analysis
- Best practice identification
- Competitive gaps requiring closure
- Differentiation opportunities
- Competitive response strategies
- Monitoring and intelligence framework
""",
"PROMPT_13_GAPS": """Identify gaps between current and desired future state across the 7S elements.
INPUTS NEEDED:
- Vision for future state (3-5 years): [DESCRIBE]
- Current state assessment: [SUMMARIZE BY S]
- Strategic objectives: [LIST]
- Performance targets: [SPECIFY]
- Market/industry trends: [IDENTIFY]
- Stakeholder expectations: [DESCRIBE]
Analyze:
1. Current state baseline for each S
2. Future state requirements per S
3. Gap magnitude and complexity
4. Interdependency impact analysis
5. Resource requirements for gap closure
6. Timeline and sequencing needs
OUTPUT:
- Current vs. future state comparison table
- Gap severity assessment (critical/high/medium/low)
- Gap closure difficulty matrix
- Investment requirements estimate
- Transformation roadmap with phases
- Quick wins vs. long-term initiatives
- Success metrics and milestones
""",
"PROMPT_14_INTEGRATION": """Combine McKinsey 7S insights with SWOT or PESTLE analysis.
INPUTS NEEDED:
- 7S assessment summary: [PROVIDE]
- SWOT analysis: [IF AVAILABLE]
- PESTLE factors: [IF AVAILABLE]
- Strategic options under consideration: [LIST]
- Risk factors identified: [LIST]
- Opportunity areas: [DESCRIBE]
Integrate analyses to:
1. Map external factors to internal capabilities
2. Identify strategic option feasibility
3. Assess implementation capabilities
4. Determine competitive positioning
5. Evaluate risk mitigation capacity
6. Prioritize strategic initiatives
OUTPUT:
- Integrated strategy framework
- Strategic option evaluation matrix
- Capability-opportunity alignment map
- Risk-readiness assessment
- Strategic initiative prioritization
- Implementation feasibility scores
- Integrated dashboard design
""",
"PROMPT_15_EXEC_SUMMARY": """Create an executive-level summary of McKinsey 7S analysis with actionable recommendations.
INPUTS NEEDED:
- Full 7S analysis results: [PROVIDE KEY FINDINGS]
- Strategic context and urgency: [DESCRIBE]
- Available resources: [BUDGET/CAPACITY]
- Board/Executive priorities: [LIST]
- Key stakeholder concerns: [IDENTIFY]
- Success criteria: [DEFINE]
Synthesize:
1. Critical insights from 7S analysis
2. Top 3-5 strategic imperatives
3. Quick wins vs. transformational changes
4. Investment requirements and ROI
5. Risk assessment and mitigation
6. Implementation timeline
OUTPUT:
- 2-page executive summary
- Visual 7S alignment dashboard
- Top 10 recommendations ranked by impact
- Investment and resource requirements
- 90-day, 6-month, and 1-year action plans
- Success metrics and governance model
- Key risks and mitigation strategies
- Next steps and decision requirements
""",
"MEGA_PROMPT": """Act as a senior McKinsey consultant conducting a comprehensive 7S Framework analysis. I need a complete organizational assessment with actionable insights.
[Use all context provided above and infer reasonable assumptions where N/A.]
DELIVERABLES:
- Executive Summary; Detailed Analysis by S; Integrated Findings; Strategic Recs; Implementation Plan; Appendices.
"""
}

# Map bracketed placeholders to ORG fields
PLACEHOLDER_MAP = {
    "[NAME]": ORG.get("name",""),
    "[INDUSTRY]": ORG.get("industry",""),
    "[NUMBER]": ORG.get("size_employees",""),
    "[REVENUE]": ORG.get("annual_revenue",""),
    "[LOCATIONS]": ORG.get("locations",""),
    "[BRIEF DESCRIPTION]": ORG.get("current_context",""),
    "[LIST 3-5]": ORG.get("key_challenges",""),
    "[DESCRIBE]": ORG.get("recent_changes",""),
    "[PROVIDE]": ORG.get("strategy_statement",""),
    "[LIST]": ORG.get("strategic_goals",""),
    "[STATEMENT]": ORG.get("value_proposition",""),
    "[DESCRIBE HIERARCHY]": ORG.get("org_chart",""),
    "[NUMBERS]": ORG.get("span_of_control",""),
    "[IF AVAILABLE]": "",
    "[AVERAGES]": "",
    "[BREAKDOWN]": "",
    "[DATA]": "",
    "[POSITION VS MARKET]": "",
    "[ASSESS 1-5]": "",
    "[CURRENT STATE]": "",
    "[ASSESS]": "",
    "[SUMMARIZE]": "",
    "[ANALYZE]": "",
    "[IDENTIFY]": "",
    "[SPECIFY]": ""
}

def materialize_prompt(key: str) -> str:
    text = PROMPTS[key]
    for k,v in PLACEHOLDER_MAP.items():
        text = text.replace(k, str(v if v else f"N/A: {k}"))
    return text

# ---------------------------
# 4) OLLAMA CHAT HELPERS
# ---------------------------
TEMPERATURE, TOP_P, REPEAT_PENALTY, TIMEOUT_S = 0.4, 0.9, 1.1, 120

def ask_llm(model: str, sys: str, user: str) -> str:
    if USE_GENERATE:
        prompt = f"<<SYS>>\n{sys}\n<</SYS>>\n\n{user}"
        r = requests.post(GEN, json={
            "model": model, "prompt": prompt, "stream": False,
            "options": {"temperature": TEMPERATURE, "top_p": TOP_P, "repeat_penalty": REPEAT_PENALTY}
        }, timeout=TIMEOUT_S)
        r.raise_for_status()
        return (r.json().get("response") or "").strip()
    else:
        r = requests.post(CHAT, json={
            "model": model,
            "messages": [{"role":"system","content":sys},{"role":"user","content":user}],
            "stream": False,
            "options": {"temperature": TEMPERATURE, "top_p": TOP_P, "repeat_penalty": REPEAT_PENALTY}
        }, timeout=TIMEOUT_S)
        r.raise_for_status()
        return ((r.json().get("message") or {}).get("content") or "").strip()

def synthesize_consensus(primary_model: str, prompt_name: str, per_model: Dict[str,str]) -> str:
    sys = "You are a senior consultant. Concisely synthesize multiple drafts into one coherent, non-redundant answer."
    user = f"""Prompt: {prompt_name}
Combine the following {len(per_model)} model answers into one clear, structured response.
- Keep the best points; remove repetition.
- If answers conflict, state the trade-off and your recommendation.
- Use short sections and bullets.

=== ANSWERS ===
{json.dumps(per_model, indent=2, ensure_ascii=False)}
"""
    return ask_llm(primary_model, sys, user)

# ---------------------------
# 5) RUN SELECTED PROMPTS ACROSS MODELS
# ---------------------------
def run_prompt_across_models(prompt_key: str, models: List[str]) -> Dict[str,str]:
    sys = "You are precise and to-the-point."
    usr = materialize_prompt(prompt_key)
    out = {}
    for m in models:
        try:
            txt = ask_llm(m, sys, usr)
            out[m] = txt
            print(f"  ✓ {prompt_key} via {m}: {len(txt)} chars")
        except Exception as e:
            out[m] = f"(error from {m}: {e})"
            print(f"  ✗ {prompt_key} via {m}: {e}")
    return out

results = {}
consensus = {}
print("\n🧠 Running prompts:", ", ".join(RUN_PROMPTS))
for key in RUN_PROMPTS:
    per_model = run_prompt_across_models(key, available)
    results[key] = per_model
    consensus[key] = synthesize_consensus(PRIMARY_MODEL, key, per_model)

# ---------------------------
# 6) SAVE OUTPUTS
# ---------------------------
bundle = {
    "metadata": {
        "generated_at_utc": datetime.datetime.now(datetime.timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ"),
        "ollama_host": BASE,
        "models": available,
        "primary_model": PRIMARY_MODEL
    },
    "org_inputs": ORG,
    "raw_per_prompt": results,
    "consensus_per_prompt": consensus
}

with open(BUNDLE_JSON, "w", encoding="utf-8") as f:
    json.dump(bundle, f, indent=2, ensure_ascii=False)

with open(REPORT_MD, "w", encoding="utf-8") as f:
    f.write(f"# 7S Report — Consensus (models: {', '.join(available)})\n\n")
    f.write("## Organization\n")
    f.write("```json\n" + json.dumps(ORG, indent=2, ensure_ascii=False) + "\n```\n\n")
    for key in RUN_PROMPTS:
        f.write(f"## {key}\n\n")
        f.write(consensus.get(key,"").strip() + "\n\n")
        f.write("<details><summary>Model answers</summary>\n\n")
        f.write("```json\n" + json.dumps(results.get(key,{}), indent=2, ensure_ascii=False) + "\n```\n")
        f.write("</details>\n\n")

print("\n✅ Done.")
print("JSON:", BUNDLE_JSON)
print("MD  :", REPORT_MD)
# ==========================================================================================


🔧 Installing Ollama…
🚀 Starting ollama serve…
⏳ Waiting for Ollama to become ready.
📥 Pulling models (best-effort): mistral, llama3, phi3
  ✅ mistral ready
  ✅ llama3 ready
  ✅ phi3 ready
✅ Using models: mistral, llama3, phi3 (primary: mistral)
ℹ️ Using /api/generate (chat failed).

🧠 Running prompts: PROMPT_1_FULL_7S, PROMPT_2_STRATEGY, PROMPT_3_STRUCTURE, PROMPT_4_SYSTEMS, PROMPT_5_SHARED_VALUES, PROMPT_6_SKILLS, PROMPT_7_STYLE, PROMPT_8_STAFF, PROMPT_9_ALIGNMENT, PROMPT_10_CHANGE, PROMPT_11_DIGITAL, PROMPT_12_BENCH, PROMPT_13_GAPS, PROMPT_14_INTEGRATION, PROMPT_15_EXEC_SUMMARY, MEGA_PROMPT
  ✓ PROMPT_1_FULL_7S via mistral: 3982 chars
  ✓ PROMPT_1_FULL_7S via llama3: 6251 chars
  ✓ PROMPT_1_FULL_7S via phi3: 8223 chars
  ✓ PROMPT_2_STRATEGY via mistral: 4267 chars
  ✓ PROMPT_2_STRATEGY via llama3: 4140 chars
  ✓ PROMPT_2_STRATEGY via phi3: 4401 chars
  ✓ PROMPT_3_STRUCTURE via mistral: 4437 chars
  ✓ PROMPT_3_STRUCTURE via llama3: 4324 chars
  ✓ PROMPT_3_STRUCTURE via phi3: 5608 char